## Esercizio

### Esercizio 4 gennaio 2023

Si estenda il linguaggio MiniCaml con un costrutto per la definizione di funzioni ricorsive limitate,
ossia funzioni che si possono chiamare ricorsivamente per un numero limitato di volte. Il limite deve essere
specificato al momento della definizione della funzione. Quando si va ad applicare la funzione, se, durante la
sua esecuzione, il numero delle chiamate ricorsive che vengono effettuate supera il limite, l’interprete termina
sollevando un’eccezione. Se invece il numero delle chiamate ricorsive non raggiunge il limite, l’interprete
completa correttamente il calcolo

Classifichiamo gli oggetti di interesse in tre categorie:
- Espressioni (o sintassi) MiniCaml (tipo `exp`)
- Valori MiniCaml (tipo `evT`)
- Valori OCaml

Nuova espressione `LetRecLim`. Nuovo valore per la chiusura `RecLimClosure`.

### Esercizio interprete 3
Una funzione (unaria e non ricorsiva) a dominio finito è una funzione che è definita solo per un numero
finito di elementi. Ad esempio si consideri la seguente funzione con una sintassi nello stile di OCaml
```
fun y -> 50 + y for y in [0; 1; 2; 3; 4];;
```
La funzione sum è definita solamente per valori del parametro attuale che appartengono all’insieme {0, 1, 2, 3, 4},
insieme che è calcolato al momento della definizione della funzione stessa.

### Espressioni

In [2]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TRecLimClosure | TClosureFinita | TUnBound

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* Costrutto LetRecLim *)
    | LetRecLim of ide * ide * int * exp * exp
    (* Funzione a dominio finito *)
    | FunFinita of ide * exp * int list
;;

type ide = string


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TRecLimClosure
  | TClosureFinita
  | TUnBound


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | LetRecLim of ide * ide * int * exp * exp
  | FunFinita of ide * exp * int list


### Tipi esprimibili

In [3]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int  
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env
    | RecClosure of ide * ide * exp * evT env
    | RecLimClosure of ide * ide * int * exp * evT env
    | ClosureFinita of ide * exp * int list * evT env
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | RecLimClosure of ide * ide * int * exp * evT env
  | ClosureFinita of ide * exp * int list * evT env
  | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


### Typechecking

In [4]:
(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | RecLimClosure(_) -> TRecLimClosure
    | ClosureFinita(_) -> TClosureFinita
    | UnBound -> TUnBound

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TClosureFinita -> (match y with
                   | ClosureFinita(_) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TRecLimClosure -> (match y with
                     | RecLimClosure(_) -> true
                     | _ -> false
                     )
    | TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )


val getType : evT -> tname = <fun>


val typecheck : tname * evT -> bool = <fun>


### Interprete

In [5]:
(* Errori a runtime *)
exception RuntimeError of string

(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")


let rec multibind (s: evT env) (xs: ide list) (vals: evT list) : evT env =
    match (xs, vals) with
    | [], [] -> s
    | (x::xs1), (v::vals1) -> multibind (bind s x v) xs1 vals1
    | _ -> failwith "Error"

let multibind s xs vals =
    List.fold_left (fun acc (x, v) -> bind acc x v) s (List.combine xs vals)

let multibind s xs vals =
    List.fold_left2 bind s xs vals

exception RuntimeError of string


val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


...

val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


val multibind : evT env -> ide list -> evT list -> evT env = <fun>


val multibind : evT env -> ide list -> evT list -> evT env = <fun>


val multibind : evT env -> ide list -> evT list -> evT env = <fun>


In [8]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(fname, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv fname (RecClosure(fname, arg, fbody, fDecEnv)) in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            (* Nuovo caso per applicare le funzioni ricorsive limitate *)
            | RecLimClosure(fname, farg, limit, fbody, fDecEnv) ->
                if limit > 0 then
                    let newenv = bind fDecEnv farg (eval eArg s) in
                    let newenv1 = bind newenv fname (RecLimClosure(fname, farg, limit - 1, fbody, fDecEnv)) in
                    eval fbody newenv1
                else
                   raise ( RuntimeError "Sorpassato limite ricorsione")
            (* Nuovo caso per applicare le funzioni a dominio finito *)
            | ClosureFinita(farg, fbody, ls, fDecEnv) ->
                let argval = eval eArg s in
                (match argval with
                | Int(v) ->
                    if List.mem v ls then
                        let newenv = bind fDecEnv farg argval in
                        eval fbody newenv
                    else
                        raise ( RuntimeError "Argomento non nel dominio")
                | _ -> raise ( RuntimeError "Wrong type")
                )
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (* LetRecLim *)
    | LetRecLim(fname, farg, limit, fbody, letbody) ->
        (* fname, farg :: ide, limit :: int, fbody, letbody :: exp *)
        (* | RecLimClosure of ide * ide * int * exp * evT env *)
        let fClosure = RecLimClosure(fname, farg, limit, fbody, s) in
        let newenv = bind s fname fClosure in
        eval letbody newenv
    (* Costruttore FunFinita *)
    | FunFinita(farg, fbody, ls) -> ClosureFinita(farg, fbody, ls, s)

val eval : exp -> evT env -> evT = <fun>


In [9]:
let test = LetRecLim(
    "fact", "n", 5,
    IfThenElse(
        IsZero (Den "n"),
        EInt 1,
        Prod(Den "n", Apply(Den "fact", Diff(Den "n", EInt 1)))),
    Apply(Den "fact", Sum(EInt 1, EInt 3))
);;

eval test emptyenv;;

val test : exp =
  LetRecLim ("fact", "n", 5,
   IfThenElse (IsZero (Den "n"), EInt 1,
    Prod (Den "n", Apply (Den "fact", Diff (Den "n", EInt 1)))),
   Apply (Den "fact", Sum (EInt 1, EInt 3)))


- : evT = Int 24
